step_1

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
import os

# ----------------------
# 1. 核心配置
# ----------------------
CONFIG = {
    "num_classes": 4,          # 4类
    "img_size": 256,           # Swin V2 Tiny 分辨率
    "batch_size": 32,          # 显存够大可以改 64
    "data_dir": "D:/IT/CODE/JUPYTER/ConvNeXt_V2/data", # 你的路径
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

print(f"当前配置: {CONFIG['num_classes']} 分类, 运行设备: {CONFIG['device']}")

# ----------------------
# 2. 数据增强 (升级版)
# ----------------------

# 【训练集】强数据增强
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(CONFIG['img_size'], scale=(0.8, 1.0)), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 【验证集 & 测试集】普通处理 (只缩放，不乱动)
val_transforms = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ----------------------
# 3. 数据加载
# ----------------------
# 加载两次：一次为了训练(强增强)，一次为了验证/测试(标准处理)
full_train_ds = datasets.ImageFolder(root=CONFIG['data_dir'], transform=train_transforms)
full_val_ds = datasets.ImageFolder(root=CONFIG['data_dir'], transform=val_transforms)

# 打印类别映射
print("类别映射:", full_train_ds.class_to_idx)

# ----------------------
# 4. 划分训练集、验证集、测试集 (8:1:1)
# ----------------------
# 计算数量
total_len = len(full_train_ds)
train_len = int(0.7 * total_len)
val_len = int(0.15 * total_len)
test_len = total_len - train_len - val_len # 剩下的给测试集

# 设定种子确保切分一致
seed = torch.Generator().manual_seed(42)

# A. 切出训练集 (使用 full_train_ds，带强增强)
train_dataset, _, _ = random_split(full_train_ds, [train_len, val_len, test_len], generator=seed)

seed = torch.Generator().manual_seed(42) # 重置种子，位置对齐

# B. 切出验证集和测试集 (使用 full_val_ds，带标准处理)
_, val_dataset, test_dataset = random_split(full_val_ds, [train_len, val_len, test_len], generator=seed)

# ----------------------
# 5. 创建加载器
# ----------------------
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0, pin_memory=True)
# 测试集用于最后写报告，不需要 shuffle
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0, pin_memory=True)

print(f"✅ 数据加载完毕！")
print(f"训练集: {len(train_dataset)} (强增强)")
print(f"验证集: {len(val_dataset)} (标准)")
print(f"测试集: {len(test_dataset)} (标准 - 用于最终报告)")

当前配置: 4 分类, 运行设备: cuda
检测到的类别映射: {'class 0': 0, 'class 1': 1, 'class 2': 2, 'class 3': 3}
数据加载完毕！训练集: 5646 张, 验证集: 1412 张


step_2

In [ ]:
import timm
import torch
from peft import LoraConfig, get_peft_model

# ----------------------
# Step 2: 定义 LoRA 模型
# ----------------------
def get_lora_model(num_classes):
    print(f"正在构建 Swin V2 + LoRA 模型 (Class={num_classes})...")
    
    # 1. 加载基础模型 (预训练必须开启)
    base_model = timm.create_model(
        'swinv2_tiny_window16_256', 
        pretrained=True, 
        num_classes=num_classes
    )
    
    # 2. 配置 LoRA
    # target_modules: 针对 Swin 的注意力层进行微调
    # modules_to_save=["head"]: 【关键】分类头必须完全解冻，因为它要适应你的4分类
    peft_config = LoraConfig(
        r=16,             # 秩大小，控制参数量
        lora_alpha=16,    # 缩放系数
        target_modules=["qkv", "proj", "fc1", "fc2"], 
        lora_dropout=0.1,
        bias="none",
        modules_to_save=["head"] 
    )
    
    # 3. 包装模型
    model = get_peft_model(base_model, peft_config)
    
    # 打印一下我们只训练了多少参数
    model.print_trainable_parameters()
    
    model = model.to(CONFIG['device'])
    return model

# 初始化模型
model = get_lora_model(CONFIG['num_classes'])

step_3

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import sys

# ----------------------
# 1. 准备训练工具 (LoRA 版)
# ----------------------
# 使用标签平滑，增加模型泛化能力
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# 【关键修改】LoRA 的参数很少，学习率需要大一点，通常用 1e-3 或 5e-4
# 原来的 1e-5 对 LoRA 来说太小了，会导致收敛不动
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

# 调度器 (余弦退火)
EPOCHS = 20 
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-5)

# 混合精度
scaler = torch.amp.GradScaler('cuda') 

# ----------------------
# 2. 定义训练函数 (保持原汁原味)
# ----------------------
def train_one_epoch(epoch_index, model, loader, optimizer, criterion, scheduler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    loop = tqdm(loader, desc=f"Epoch {epoch_index} [Train]", file=sys.stdout)
    
    for batch_idx, (images, labels) in enumerate(loop):
        images, labels = images.to(CONFIG['device']), labels.to(CONFIG['device'])
        
        optimizer.zero_grad()
        
        # 混合精度前向
        with torch.amp.autocast('cuda', dtype=torch.float16):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # 反向传播
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # 统计
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        current_lr = optimizer.param_groups[0]['lr']
        current_avg_loss = running_loss / (batch_idx + 1)
        
        # 显示当前 LR
        loop.set_postfix(loss=f"{current_avg_loss:.4f}", acc=f"{100.*correct/total:.2f}%", lr=f"{current_lr:.1e}")
        
    return running_loss / len(loader), 100. * correct / total

# ----------------------
# 3. 定义验证函数 (保持原汁原味)
# ----------------------
def validate(epoch_index, model, loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    loop = tqdm(loader, desc=f"Epoch {epoch_index} [Val  ]", file=sys.stdout)
    
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loop):
            images, labels = images.to(CONFIG['device']), labels.to(CONFIG['device'])
            
            with torch.amp.autocast('cuda', dtype=torch.float16):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            current_avg_loss = val_loss / (batch_idx + 1)
            loop.set_postfix(loss=f"{current_avg_loss:.4f}", acc=f"{100.*correct/total:.2f}%")
            
    return val_loss / len(loader), 100. * correct / total

# ----------------------
# 4. 开始训练
# ----------------------
best_acc = 0.0
print(f"🔥 开始 LoRA 微调训练... 设备: {CONFIG['device']}")

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(epoch, model, train_loader, optimizer, criterion, scheduler)
    val_loss, val_acc = validate(epoch, model, val_loader, criterion)
    
    # 更新学习率
    scheduler.step()
    
    # 保存最佳模型
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model_lora.pth") # 保存为 lora 版文件名
        print(f"--> 🚀 发现新高! 保存模型 (Val Acc: {val_acc:.2f}%)")
        
    print(f"Epoch {epoch} 结束. Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print("-" * 60)

print(f"训练完成！最高准确率: {best_acc:.2f}%")

In [ ]:
import torch
# 检查是否成功调用了 CUDA
if torch.cuda.is_available():
    print(f"✅ 显卡已就绪: {torch.cuda.get_device_name(0)}")
    # 测试一下显存写入
    x = torch.ones(1).cuda()
    print("✅ 显存写入测试通过！")
else:
    print("❌ 依然没有检测到显卡，请截图报错信息。")